In [198]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
    
import markdown
from lxml import etree
import re
from urllib.parse import urlsplit

In [2]:
downs = pd.read_csv('results-20180502-121033.csv')

In [4]:
ups = pd.read_csv('2014_ups.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
ups['dnv'] = 0
downs['dnv'] = 1

In [159]:
comments = pd.concat([ups,downs])
comments = comments.reset_index()
drop_cols = ['index','score_hidden','name','downs','subreddit_id','subreddit','controversiality','gilded','ups','distinguished','removal_reason']

comments = comments.drop(drop_cols,axis=1)

comments['body'] = comments['body'].apply(lambda x: drop_newlines(x))
comments['created_utc'] = pd.to_datetime(comments['created_utc'],unit='s')
comments['retrieved_on'] = pd.to_datetime(comments['retrieved_on'],unit='s')

comments['age_retrieved'] = comments['retrieved_on'] - comments['created_utc']

In [51]:
comments.shape

(37867, 12)

In [82]:
body_markdown = "This is an [inline link](http://google.com). This is a [non inline link][1]\r\n\r\n  [1]: http://yahoo.com"


In [127]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

def extract_md_link(post):
    doc = etree.fromstring(markdown.markdown(post))
    links = []
    for link in doc.xpath('//a'):
        links.append(link.get('href'))    
    return ' '.join(links)

def drop_newlines(post):
    return ' '.join(post.split('\n'))

In [230]:
def get_link(post):

    myString_list = [item for item in post.split(" ")]
    url_list = []
    for item in myString_list:
        try:
            a = re.search("(?P<url>https?://[^\s]+)", item) or re.search("(?P<url>www[^\s]+)", url)
            a = a.group('url')
            b = '.'.join(urlsplit(a).hostname.split('.')[-2:])
            
            url_list.append(b)
        except:
            pass

    return ' '.join(url_list)

In [231]:
comments['word_count'] = comments['body'].apply(lambda x: len(str(x).split(' ')))
comments['avg_word'] = comments['body'].apply(lambda x: avg_word(x))

comments['links'] = comments['body'].apply(lambda x: get_link(x))

In [232]:
comments['links'].value_counts()

                                                                                                                                                                                                                        34960
imgur.com                                                                                                                                                                                                                 317
wikipedia.org                                                                                                                                                                                                             143
reddit.com                                                                                                                                                                                                                123
seattle.gov                                                                                                     